In [1]:
import sys
sys.path.insert(0, '../..')
import pandas as pd
import numpy as np
from sdv.tabular import CTGAN
from sklearn.model_selection import train_test_split
from data_utils import TableProcessor, TableConfig
%load_ext autoreload
%autoreload 2

In [4]:
full_file = './data/full/cardio_full.csv'
df_full = pd.read_csv(full_file, delimiter=';')
df_full

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99993,19240,2,168,76.0,120,80,1,1,1,0,1,0
69996,99995,22601,1,158,126.0,140,90,2,2,0,0,1,1
69997,99996,19066,2,183,105.0,180,90,3,1,0,1,0,1
69998,99998,22431,1,163,72.0,135,80,1,2,0,0,0,1


# Preprocessing

In [15]:
config = TableConfig('./data/cardio.json', label='cardio')
train_file = './data/ir30/cardio_train_ir_30.csv'
df_train = pd.read_csv(train_file)
ctgan_file = './data/ir30/ctgan_samples.csv'
df_ctgan = pd.read_csv(ctgan_file)
df = pd.concat([df_train, df_ctgan])
processor = TableProcessor(config)
new_df = processor.process_dataframe(df)
np.savez('./data/ir30/cardio_ctgan_ir_30', X=new_df['features'], y=new_df['targets'])

In [28]:
df_ctgan[df_ctgan['gluc']==0]

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
3,20825,0,171,70.206033,99,61,0,0,0,1,1,1
4,18428,0,168,115.058504,85,76,0,0,0,0,0,1
5,17043,2,168,63.854642,154,75,2,0,0,0,0,1
17,14336,0,153,71.552087,125,76,2,0,0,0,1,1
19,19158,1,161,62.492468,134,75,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
22635,14812,2,180,51.611882,119,77,0,0,0,1,0,1
22636,17153,0,167,61.865326,132,73,0,0,1,0,0,1
22645,24641,0,167,62.628683,120,77,0,0,0,0,1,1
22647,21539,2,167,70.911533,122,61,2,0,0,0,1,1


# CTGAN

In [10]:
df_ir_30 = pd.read_csv('./data/ir30/cardio_train_ir_30.csv')
df_full = pd.read_csv('./data/full/cardio_train.csv')
n_samples = df_full['cardio'].sum()-df_ir_30['cardio'].sum()

In [12]:
model = CTGAN()
model.fit(df_ir_30)

/Users/broniyasinnik/PycharmProjects/bigdata/ml/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/broniyasinnik/PycharmProjects/bigdata/ml/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/broniyasinnik/PycharmProjects/bigdata/ml/lib/python3.8/site-packages/sklearn/mixture/_base.py:147: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
/Users/broniyasinnik/PycharmProjects/bigdata/ml/lib/python3.8/site-packages/sklearn/utils/valida

In [14]:
model.save('./ctgan/ir30/ctgan.pkl')

In [16]:
conditions = {
    'cardio': 1
}
new_samples = model.sample(n_samples, conditions=conditions)
new_samples

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,22538,2,198,79.201161,75,76,1,1,0,0,0,1
1,19377,2,163,155.140069,122,72,0,1,1,0,0,1
2,19217,0,161,193.889785,102,75,0,2,0,0,0,1
3,20825,0,171,70.206033,99,61,0,0,0,1,1,1
4,18428,0,168,115.058504,85,76,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
22649,18620,0,167,59.845284,117,66,1,1,0,0,0,1
22650,14289,0,154,231.263500,103,77,2,1,0,0,1,1
22651,18456,0,184,64.932881,121,74,2,1,0,1,0,1
22652,14432,0,185,72.160505,89,77,1,2,1,0,0,1


In [20]:
df_ir_30[df_ir_30['cardio']==1]

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
23464,22085,1,162,80.0,160,90,1,1,0,0,1,1
23465,18126,1,165,70.0,140,90,1,1,0,0,1,1
23466,18755,2,185,108.0,160,90,1,1,1,0,1,1
23467,23434,2,170,64.0,130,80,1,1,1,0,1,1
23468,14456,2,167,73.0,140,80,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
24241,21999,2,180,95.0,140,80,2,1,0,1,1,1
24242,22787,1,158,62.0,140,90,1,1,0,0,0,1
24243,19781,1,163,69.0,120,80,2,1,0,0,1,1
24244,18293,1,165,90.0,160,100,1,1,0,0,0,1


In [17]:
new_samples.to_csv('./ctgan/ir30/samples.csv', index=False)

In [6]:
loaded = CTGAN.load('ctgab_ir_1.pkl')

# Create Imbalance

In [1]:
from datasets import TableDataset
from data_utils import DatasetImbalanced

In [3]:
train_file = './data/full/cardio_train.npz'
test_file = './data/full/cardio_test.npz'
train_data = TableDataset.from_npz(train_file, train=True)
test_data = TableDataset.from_npz(test_file, train=False)

In [4]:
ir =30
train_imb = DatasetImbalanced(train_data, imbalance_ratio=1/ir, create_complement_dataset=True)
test_imb = DatasetImbalanced(test_data, imbalance_ratio=1/ir)

In [5]:
train_imb.save_npz(f'./data/ir{ir}/cardio_train_ir_{ir}.npz')
test_imb.save_npz(f'./data/ir{ir}/cardio_test_ir_{ir}.npz')
np.savez(f'./data/ir{ir}/cardio_train_complement.npz', 
         X=train_imb.complement_dataset.data.numpy(),
         y=train_imb.complement_dataset.target.numpy()
        )

In [6]:
print("tain majority size:", train_imb.num_majority)
print("tain minority size:", train_imb.num_minority)
print("test majority size:", test_imb.num_majority)
print("test minority size:", test_imb.num_minority)

tain majority size: 23464
tain minority size: 782
test majority size: 11557
test minority size: 385


In [11]:
train_imb_file = './data/ir30/cardio_train_ir_30.npz'
arr = np.load(train_imb_file)
X, y = arr["X"], arr["y"]
y.sum()

782.0

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

In [13]:
np.savez('./data/ir30/cardio_train.npz', X=X_train, y=y_train)
np.savez('./data/ir30/cardio_holdout.npz', X=X_test, y=y_test)

In [14]:
print("train size:", X_train.shape)
print("minority size in train:", y_train.sum())
print("test size:", X_test.shape)
print("test minority size:", y_test.sum())

train size: (21821, 15)
minority size in train: 704.0
test size: (2425, 15)
test minority size: 78.0


In [9]:
(2425-78)/78

30.08974358974359

In [5]:
data = np.load('./data/ir30/cardio_holdout.npz')
list(data.keys())

['X_ho', 'y_ho']